<a href="https://colab.research.google.com/github/banakonwee/Credit-Risk-Analysis.ipynb/blob/main/5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kivy

In [ ]:
from kivy.lang import Builder
from kivy.uix.tabbedpanel import TabbedPanel
from kivy.core.window import Window
from kivy.app import App
from kivy.uix.popup import Popup
from kivy.uix.label import Label
import sqlite3
import os

DB_NAME = "credit_management.db"

Builder.load_string('''
<CreditManagementSystem>:
    size_hint: 1, 1
    pos_hint: {'center_x': 0.5, 'center_y': 0.5}
    do_default_tab: False

    TabbedPanelItem:
        text: 'Customers'
        BoxLayout:
            orientation: 'vertical'
            padding: 10
            spacing: 10

            BoxLayout:
                size_hint_y: None
                height: 30
                Label:
                    text: 'Name:'
                TextInput:
                    id: name_input
            BoxLayout:
                size_hint_y: None
                height: 30
                Label:
                    text: 'Email:'
                TextInput:
                    id: email_input
            BoxLayout:
                size_hint_y: None
                height: 30
                Label:
                    text: 'Phone:'
                TextInput:
                    id: phone_input
            BoxLayout:
                size_hint_y: None
                height: 30
                Label:
                    text: 'Address:'
                TextInput:
                    id: address_input
            BoxLayout:
                size_hint_y: None
                height: 40
                spacing: 5
                Button:
                    text: 'Add Customer'
                    on_press: root.add_customer()
                Button:
                    text: 'Clear'
                    on_press: root.clear_customer_fields()

            RecycleView:
                viewclass: 'Label'
                data: [{'text': str(x)} for x in root.customers_data]
                RecycleBoxLayout:
                    size_hint_y: None
                    height: self.minimum_height
                    orientation: 'vertical'

    TabbedPanelItem:
        text: 'Loans'
        BoxLayout:
            orientation: 'vertical'
            padding: 10
            spacing: 10

            BoxLayout:
                size_hint_y: None
                height: 30
                Label:
                    text: 'Customer ID:'
                TextInput:
                    id: customer_id_input
            BoxLayout:
                size_hint_y: None
                height: 30
                Label:
                    text: 'Loan Amount:'
                TextInput:
                    id: loan_amount_input
            BoxLayout:
                size_hint_y: None
                height: 30
                Label:
                    text: 'Loan Term:'
                TextInput:
                    id: loan_term_input
            BoxLayout:
                size_hint_y: None
                height: 30
                Label:
                    text: 'Income:'
                TextInput:
                    id: income_input
            BoxLayout:
                size_hint_y: None
                height: 40
                spacing: 5
                Button:
                    text: 'Add Loan'
                    on_press: root.add_loan()
                Button:
                    text: 'Clear'
                    on_press: root.clear_loan_fields()

            RecycleView:
                viewclass: 'Label'
                data: [{'text': str(x)} for x in root.loans_data]
                RecycleBoxLayout:
                    size_hint_y: None
                    height: self.minimum_height
                    orientation: 'vertical'
''')

class CreditManagementSystem(TabbedPanel):
    customers_data = []
    loans_data = []

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.create_database()
        Window.bind(on_keyboard=self.keyboard_handler)
        self.load_customers()
        self.load_loans()

    def create_database(self):
        """Create database tables if not exist"""
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Customers (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT,
                email TEXT,
                phone TEXT,
                address TEXT
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Loans (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                customer_id INTEGER,
                loan_amount REAL,
                loan_term INTEGER,
                income REAL,
                FOREIGN KEY(customer_id) REFERENCES Customers(id)
            )
        ''')
        conn.commit()
        conn.close()

    def add_customer(self):
        name = self.ids.name_input.text
        email = self.ids.email_input.text
        phone = self.ids.phone_input.text
        address = self.ids.address_input.text

        if not all([name, email, phone, address]):
            self.show_popup("Error", "All fields are required!")
            return

        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        cursor.execute("INSERT INTO Customers (name, email, phone, address) VALUES (?, ?, ?, ?)",
                       (name, email, phone, address))
        conn.commit()
        conn.close()
        self.load_customers()
        self.clear_customer_fields()

    def load_customers(self):
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM Customers")
        self.customers_data = cursor.fetchall()
        conn.close()

    def clear_customer_fields(self):
        self.ids.name_input.text = ""
        self.ids.email_input.text = ""
        self.ids.phone_input.text = ""
        self.ids.address_input.text = ""

    def add_loan(self):
        customer_id = self.ids.customer_id_input.text
        loan_amount = self.ids.loan_amount_input.text
        loan_term = self.ids.loan_term_input.text
        income = self.ids.income_input.text

        if not all([customer_id, loan_amount, loan_term, income]):
            self.show_popup("Error", "All fields are required!")
            return

        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        cursor.execute("INSERT INTO Loans (customer_id, loan_amount, loan_term, income) VALUES (?, ?, ?, ?)",
                       (customer_id, loan_amount, loan_term, income))
        conn.commit()
        conn.close()
        self.load_loans()
        self.clear_loan_fields()

    def load_loans(self):
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM Loans")
        self.loans_data = cursor.fetchall()
        conn.close()

    def clear_loan_fields(self):
        self.ids.customer_id_input.text = ""
        self.ids.loan_amount_input.text = ""
        self.ids.loan_term_input.text = ""
        self.ids.income_input.text = ""

    def show_popup(self, title, message):
        content = Label(text=message)
        popup = Popup(title=title, content=content, size_hint=(None, None), size=(400, 200))
        popup.open()

    def keyboard_handler(self, instance, key, *args):
        if key == 27:  # Android back button
            return True

class CreditApp(App):
    Kv_file = None # Changed 'none' to 'None'
    def build(self):
        return CreditManagementSystem()